# California Air Resources Board (CARB)

In [29]:
import requests
import pandas as pd
from io import BytesIO
from bs4 import BeautifulSoup

### Parameters

In [5]:
URL = "https://ww2.arb.ca.gov/mrr-data"

headers = {
    "Authority": "ww2.arb.ca.gov",
    "Method": "GET",
    "Path": "/mrr-data",
    "Scheme": "https",
    "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7",
    "Accept-Encoding": "gzip, deflate, br",
    "Accept-Language": "en-US,en;q=0.9",
    "Cache-Control": "max-age=0",
    "If-None-Match": "1689111602",
    "Sec-Ch-Ua": '"Not.A/Brand";v="8", "Chromium";v="114", "Google Chrome";v="114"',
    "Sec-Ch-Ua-Mobile": "?0",
    "Sec-Ch-Ua-Platform": '"Windows"',
    "Sec-Fetch-Dest": "document",
    "Sec-Fetch-Mode": "navigate",
    "Sec-Fetch-Site": "cross-site",
    "Sec-Fetch-User": "?1",
    "Upgrade-Insecure-Requests": "1",
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36"
}


### Tests

In [6]:
r = requests.get(URL, headers=headers)

In [17]:
soup = BeautifulSoup(r.content, "html.parser")
soup
# Get all hrefs and their text
href_dicts = []
hrefs = soup.find_all("a")
for href in hrefs:
    href_dict = {}
    text=  href.text
    url = href.get("href")

    if "xls" in url:     
        href_dict["name"] = text
        href_dict["url"] = f"https:{url}" 

        href_dicts.append(href_dict)
    

In [20]:
urls_df = pd.DataFrame.from_dict(href_dicts)

### Access one of the files

In [26]:
xlsx_url = urls_df['url'][0]

# Get data from first url
df = pd.read_excel(xlsx_url)

c:\Users\fdmol\AppData\Local\Programs\Python\Python311\Lib\site-packages\openpyxl\worksheet\_reader.py:329: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)


In [30]:
response = requests.get(xlsx_url)
xls_file = pd.ExcelFile(BytesIO(response.content))
# Create a dictionary of DataFrames, with sheet name as key
dataframes = {sheet_name: xls_file.parse(sheet_name) for sheet_name in xls_file.sheet_names}

c:\Users\fdmol\AppData\Local\Programs\Python\Python311\Lib\site-packages\openpyxl\worksheet\_reader.py:329: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)
c:\Users\fdmol\AppData\Local\Programs\Python\Python311\Lib\site-packages\openpyxl\worksheet\_reader.py:329: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)


In [37]:
ghg_data = dataframes['2021 GHG Data']
# Remove rows with several NaNs
ghg_data = ghg_data.dropna(thresh=10)
# First row is the header
ghg_data.columns = ghg_data.iloc[0]
# Remove rows
longbeach_df = ghg_data.loc[ghg_data.loc[:, 'City'] == 'Long Beach']
longbeach_df

8,NaN,ARB ID,Facility Name,Report\nYear,NaN,"Total CO2e \n(combustion, process, vented, and supplier)",AEL,NaN,Emitter CO2e from Non-Biogenic Sources and CH4 and N2O from Biogenic Fuels,Emitter CO2 from Biogenic Fuels,...,Emissions Data,Product Data,Verification Body,NaN,City,State,Zip Code,North American Industry Classification System (NAICS) \nCode and Description,U.S.EPA/ARB Subparts,Industry Sector
18,NaN,101321,"AES Alamitos, LLC",2021,NaN,1477435.567312,No,NaN,1477435.567312,0,...,Positive,NaN,Ashworth Leininger Group,NaN,Long Beach,CA,90803,221112 - Fossil Fuel Electric Power Generation,"C,D",In-State Electricity Generation
56,NaN,5003,Long Beach Gas & Oil Dept (LBGO),2021,NaN,511924.130656,No,NaN,0,0,...,Positive,NaN,Locus Technologies,NaN,Long Beach,CA,90806,221210 - Natural Gas Distribution,NN,"Supplier of Natural Gas, NGL, or LPG"
99,NaN,100002,Calciner,2021,NaN,285809.907123,No,NaN,285809.907123,0,...,Positive,Positive,"Tetra Tech, Inc. and Subsidiaries",NaN,Long Beach,CA,90813,324199 - All Other Petroleum and Coal Products...,"C,Y",Other Combustion Source
122,NaN,104072,California Resources Production Corporation - ...,2021,NaN,220714.719382,No,NaN,220714.719382,0,...,Positive,Positive,"Tetra Tech, Inc. and Subsidiaries",NaN,Long Beach,CA,90802,211111 - Crude Petroleum and Natural Gas Extra...,"C,W",Oil and Gas Production
300,NaN,101697,"Gold Bond Building Products, LLC - Long Beach",2021,NaN,36091.208986,No,NaN,36091.208986,0,...,Positive,Positive,SCS Engineers,NaN,Long Beach,CA,90813,327420 - Gypsum Product Manufacturing,C,Other Combustion Source
408,NaN,100208,Long Beach Generating Station,2021,NaN,13954.070074,No,NaN,13954.070074,0,...,Positive,NaN,Locus Technologies,NaN,Long Beach,CA,90802,221112 - Fossil Fuel Electric Power Generation,D,In-State Electricity Generation
411,NaN,100187,Los Angeles Department of Water & Power-Haynes...,2021,NaN,1053475.324852,No,NaN,1053475.324852,0,...,Positive,NaN,Ashworth Leininger Group,NaN,Long Beach,CA,90803,221112 - Fossil Fuel Electric Power Generation,"C,D",In-State Electricity Generation
697,NaN,100063,Southeast Resource Recovery Facility (SERRF),2021,NaN,339876.424422,No,NaN,134818.424422,205058,...,Positive,NaN,"Trinity Consultants, Inc.",NaN,Long Beach,CA,90802,562213 - Solid Waste Combustors and Incinerators,C,In-State Electricity Generation
